In [2]:
import pandas as pd
from clingo import Control
import ast

students_df =pd.read_csv('../data/studenten.csv')
mentors_df =pd.read_csv('../data/mentoren.csv')

education_mapping = {
    'Associate': 1,
    'Bachelor': 2, 
    'Master': 3,
    'PhD': 4
}

def generate_asp_facts():
    facts = []
    
    for _, row in students_df.iterrows():
        student_id = f"s{row.name}"  
        facts.append(f"student({student_id}).")
        
        edu_level = education_mapping[row['Opleidingsniveau']]
        facts.append(f"education({student_id}, {edu_level}).")
        
        subject = row['Onderwerp'].lower().replace(' ', '_')
        facts.append(f"expertise({student_id}, {subject}).")
    
    for _, row in mentors_df.iterrows():
        mentor_id = f"m{row.name}"  
        facts.append(f"mentor({mentor_id}).")
        
        edu_level = education_mapping[row['Opleidingsniveau']]
        facts.append(f"education({mentor_id}, {edu_level}).")
        
        subjects = ast.literal_eval(row['Onderwerpen'])
        for subject in subjects:
            subject_clean = subject.lower().replace(' ', '_')
            facts.append(f"expertise({mentor_id}, {subject_clean}).")
    
    return "\n".join(facts)

asp_facts = generate_asp_facts()

ctl = Control()
ctl.add("base", [], f"""
{asp_facts}

% Matching constraints
valid_mentor(S, M) :- student(S), mentor(M), expertise(M, Subj), expertise(S, Subj), education(M, EM), education(S, ES), EM > ES.

% Allow 0 or 1 mentor per student 
{{ match_mentor(S, M) : valid_mentor(S, M) }} <= 1 :- student(S).

% Maximize the number of matched students 
#maximize {{ 1,S : match_mentor(S,M) }}.

% Output
#show match_mentor/2.
""")

ctl.ground([("base", [])])

def print_solution(model):
    print("\n=== MENTOR MATCHING SOLUTION ===")
    matches = []
    
    for symbol in model.symbols(shown=True):
        if symbol.name == "match_mentor":
            student_id = symbol.arguments[0].name
            mentor_id = symbol.arguments[1].name
            
            student_idx = int(student_id[1:])  
            mentor_idx = int(mentor_id[1:])    
            
            student_name = f"{students_df.iloc[student_idx]['Voornaam']} {students_df.iloc[student_idx]['Achternaam']}"
            mentor_name = f"{mentors_df.iloc[mentor_idx]['Voornaam']} {mentors_df.iloc[mentor_idx]['Achternaam']}"
            student_subject = students_df.iloc[student_idx]['Onderwerp']
            
            matches.append((student_name, mentor_name, student_subject))
    
    print("\nMentor Assignments:")
    for student, mentor, subject in sorted(matches):
        print(f"  {student} ({subject}) → {mentor}")
    
    total_students = len(students_df)
    matched_students = len(matches)
    unmatched_students = total_students - matched_students
    
    print(f"  Total students: {total_students}")
    print(f"  Successfully matched: {matched_students}")
    print(f"  Unmatched: {unmatched_students}")
    print(f"  Success rate: {matched_students/total_students*100:.1f}%")
    print("="*40)

result = ctl.solve(on_model=print_solution)
print(f"\nSolver result: {result}")



=== MENTOR MATCHING SOLUTION ===

Mentor Assignments:
  Total students: 100
  Successfully matched: 0
  Unmatched: 100
  Success rate: 0.0%

=== MENTOR MATCHING SOLUTION ===

Mentor Assignments:
  Maja Fortuyn (Design Science Research) → Ryan van Arkel
  Total students: 100
  Successfully matched: 1
  Unmatched: 99
  Success rate: 1.0%

=== MENTOR MATCHING SOLUTION ===

Mentor Assignments:
  Isabelle van de Elzas (Design Science Research) → Chloé van Oostendorp
  Maja Fortuyn (Design Science Research) → Anne Köster
  Total students: 100
  Successfully matched: 2
  Unmatched: 98
  Success rate: 2.0%

=== MENTOR MATCHING SOLUTION ===

Mentor Assignments:
  Isabelle van de Elzas (Design Science Research) → Ryan van Arkel
  Maja Fortuyn (Design Science Research) → Isa Maas
  Nick Kolster (Design Science Research) → Ryan van Arkel
  Total students: 100
  Successfully matched: 3
  Unmatched: 97
  Success rate: 3.0%

=== MENTOR MATCHING SOLUTION ===

Mentor Assignments:
  Isabelle van de Elz